Import libs

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
#switching off scientific notation
#pd.options.display.float_format = '{:.10f}'.format
#np.set_printoptions(suppress=True)

Reading csv file and renaming columns appropriately

In [6]:
df = pd.read_csv('Sweden Load Data 2005-2017.csv')
df.rename({'cet_cest_timestamp':'time','SE_load_actual_tso':'load'},axis='columns',inplace=True)

Converting time columns to datetime and extracting weekday putting it to new column in order to take count of days for RL agent to extract rewards in time.

In [7]:
df['time'] = pd.to_datetime(df['time'],errors='ignore', utc=True)
df['weekday'] = df['time'].dt.weekday

In [8]:
array_loads = df['load']
array_weekdays = df['weekday']

In [9]:
class Env:
    def __init__(self,array_loads,array_weekdays,charge=4,action=None):
        ###################
        self.array_loads = array_loads
        self.array_weekdays=array_weekdays
        self.charge = charge
        self.action = action
        ###################
        self.load_idx = 0
        self.day_idx = 0
        self.collection_actual_load = []
        self.collection_rewards = []
        self.actual_load=None
        self.reward = 0
        self.episode=0
    def increase_load_counter(self):
        self.load_idx+=1
        
    def increase_day_counter(self):
        self.day_idx+=1
        
    def generate_input_load(self):
        self.input_load = self.array_loads[self.load_idx]
        
    def calculate_actual_load(self):
        if self.action==None:
            self.actual_load = self.input_load
            self.collection_actual_load.append(self.input_load)
            self.increase_load_counter()
            
        elif self.action == 'discharge' and self.charge!=0:
            if self.input_load >= self.collection_actual_load[self.load_idx-1]:
                self.actual_load = self.collection_actual_load[self.load_idx-1]
                self.collection_actual_load.append(self.actual_load)
                self.increase_load_counter()
            else:
                self.actual_load = self.input_load
                self.collection_actual_load.append(self.actual_load)
                self.increase_load_counter()
            self.charge-=1
            
        elif self.action == 'charge' and self.charge!=4:
            if self.input_load <= self.collection_actual_load[self.load_idx-1]:
                self.actual_load = self.collection_actual_load[self.load_idx-1]
                self.collection_actual_load.append(self.actual_load)
                self.increase_load_counter()
            else:
                self.actual_load = self.input_load
                self.collection_actual_load.append(self.actual_load)
                self.increase_load_counter()
            self.charge+=1
        else:
            self.actual_load = self.input_load
            self.collection_actual_load.append(self.actual_load)
            self.increase_load_counter()
            
    def calculate_reward(self):
        self.increase_day_counter()
        #check whether we have crossed to the next day
        if self.array_weekdays[self.day_idx] == self.array_weekdays[self.day_idx-1]:
            self.reward = 0
            self.collection_rewards.append(self.reward)
        else:
            last_idx = self.collection_actual_load.index(self.collection_actual_load[-2])
            start_idx = last_idx-23
            #cost function
            #self.reward = 5+0.5*max(self.collection_actual_load[start_idx:-1])+0.05*(max(self.collection_actual_load[start_idx:-1])**2)
            self.reward = 5+0.5*max(self.collection_actual_load[-24:])+0.05*(max(self.collection_actual_load[-24:])**2)
            self.collection_rewards.append(self.reward)
            self.episode+=1

In [10]:
class Agent:    
    def __init__(self,action_space=np.array(['charge','discharge','wait'])):
        self.action_space = action_space
    
    def take_action(self,load,reward):
        candidate_action = np.random.choice(self.action_space, 1)[0]
        return candidate_action

In [11]:
agent_test = Agent()
env1 = Env(array_loads,array_weekdays)
env1.generate_input_load()
print('Input load:',env1.input_load)
env1.calculate_actual_load()
print('Actual load:',env1.actual_load)
print('All actual loads:',env1.collection_actual_load)
print('')


for i in range(24):
    env1.action=agent_test.take_action(env1.input_load,env1.reward)
    print('Action:',env1.action)
    env1.generate_input_load()
    print('Input load:',env1.input_load)
    env1.calculate_actual_load()
    print('Actual load:',env1.actual_load)
    print('All actual loads:',env1.collection_actual_load)
    env1.calculate_reward()
    print('Reward',env1.collection_rewards)

Input load: 15991.34
Actual load: 15991.34
All actual loads: [15991.34]

Action: charge
Input load: 15673.12
Actual load: 15673.12
All actual loads: [15991.34, 15673.12]
Reward [0]
Action: charge
Input load: 15431.81
Actual load: 15431.81
All actual loads: [15991.34, 15673.12, 15431.81]
Reward [0, 0]
Action: charge
Input load: 15326.48
Actual load: 15326.48
All actual loads: [15991.34, 15673.12, 15431.81, 15326.48]
Reward [0, 0, 0]
Action: discharge
Input load: 15468.59
Actual load: 15326.48
All actual loads: [15991.34, 15673.12, 15431.81, 15326.48, 15326.48]
Reward [0, 0, 0, 0]
Action: wait
Input load: 15717.19
Actual load: 15717.19
All actual loads: [15991.34, 15673.12, 15431.81, 15326.48, 15326.48, 15717.19]
Reward [0, 0, 0, 0, 0]
Action: discharge
Input load: 15936.22
Actual load: 15717.19
All actual loads: [15991.34, 15673.12, 15431.81, 15326.48, 15326.48, 15717.19, 15717.19]
Reward [0, 0, 0, 0, 0, 0]
Action: charge
Input load: 16032.7
Actual load: 16032.7
All actual loads: [15991